# GPTZero threshold

In [ ]:
# finding gptzero threshold

import sys
import json
from tqdm import tqdm 
# sys.path.append("/Users/bobaebak/git/ai_text_detection")
sys.path.append("/mnt/data/bobae/ai_text_detection")

from models.gptzero_detector import *
from utils.text_helper import tokenize_text

# human threshold

In [ ]:
# human data

# loads a tofel dataset
with open('../dataset/human/tofel.json', "r") as f:
    h_tofel_dataset = json.load(f)

# loads an arxiv dataset
with open('../dataset/human/arxiv.json', "r") as f:
    h_arxiv_dataset = json.load(f)

# loads student essay
with open('../dataset/human/student_essay.json', "r") as f:
    h_essay_dataset = json.load(f)

# loads student computer essay
with open('../dataset/human/student_cs_essay.json', "r") as f:
    h_essay_cs_dataset = json.load(f)

In [ ]:
h_dataset = []
for i in [h_tofel_dataset, h_arxiv_dataset, h_essay_dataset, h_essay_cs_dataset]:
    h_dataset.extend(i)

len(h_dataset)

In [6]:
json_lst = []

for i, data in enumerate(tqdm(h_dataset)):
    try:
        input = data['input']
        result = detect(input)
        result['num_words'] = len(tokenize_text(input))
        json_lst.append(result)

    except Exception as e:
        print(e)
    
    if i%100 == 0:
        print(f"{i} index finished")

# save as json file
file_path = "".join('/mnt/data/bobae/ai_text_detection/dataset/human/gptzero_human.json')
print(f"saved to {file_path}...")
with open(file_path, "w") as f:
    json.dump(json_lst, f, indent=4)


  0%|                                                             | 1/2478 [00:02<2:00:00,  2.91s/it]

0 index finished


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7e69a803d5a0>>
Traceback (most recent call last):
  File "/home/bobae/miniconda3/envs/ai_detector/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
  1%|▍                                                           | 16/2478 [00:24<1:03:48,  1.55s/it]


KeyboardInterrupt: 

# AI threshold

In [7]:
# gpt data

# loads a tofel dataset
with open('../dataset/ai/gpt2medium_tofel.json', "r") as f:
    gpt_tofel_dataset = json.load(f)

# loads an arxiv dataset
with open('../dataset/ai/gpt2medium_arxiv.json', "r") as f:
    gpt_arxiv_dataset = json.load(f)

# loads student essay
with open('../dataset/ai/gpt2medium_essay.json', "r") as f:
    gpt_essay_dataset = json.load(f)

# loads student computer essay
with open('../dataset/ai/gpt2medium_essay_cs.json', "r") as f:
    gpt_essay_cs_dataset = json.load(f)

# loads a tofel dataset
with open('../dataset/ai/gpt35_tofel.json', "r") as f:
    gpt_35_tofel_dataset = json.load(f)

# loads student computer essay
with open('../dataset/ai/gpt35_essay_cs.json', "r") as f:
    gpt_35_essay_cs_dataset = json.load(f)

In [8]:
gpt_dataset = []
for i in [gpt_tofel_dataset, gpt_arxiv_dataset, gpt_essay_dataset, gpt_essay_cs_dataset,
          gpt_35_tofel_dataset, gpt_35_essay_cs_dataset]:
    gpt_dataset.extend(i)

len(gpt_dataset)

2956

In [9]:
json_lst = []

for i, data in enumerate(tqdm(gpt_dataset)):
    try:
        input = data['input']
        result = detect(input)
        result['num_words'] = len(tokenize_text(input))
        json_lst.append(result)

    except Exception as e:
        print(e)
    
    if i%100 == 0:
        print(f"{i} index finished")

# save as json file
file_path = "".join('/mnt/data/bobae/ai_text_detection/dataset/ai/gptzero_ai.json')
print(f"saved to {file_path}...")
with open(file_path, "w") as f:
    json.dump(json_lst, f, indent=4)


  0%|                                                             | 1/2956 [00:01<1:10:12,  1.43s/it]

0 index finished


  1%|▎                                                           | 17/2956 [00:24<1:10:39,  1.44s/it]

cannot convert float NaN to integer


  1%|▋                                                           | 32/2956 [00:45<1:10:56,  1.46s/it]

...

cannot convert float NaN to integer


Token indices sequence length is longer than the specified maximum sequence length for this model (1205 > 1024). Running this sequence through the model will result in indexing errors
100%|█████████████████████████████████████████████████████████▉| 2955/2956 [1:11:07<00:01,  1.43s/it]

cannot convert float NaN to integer


Token indices sequence length is longer than the specified maximum sequence length for this model (1357 > 1024). Running this sequence through the model will result in indexing errors
100%|██████████████████████████████████████████████████████████| 2956/2956 [1:11:09<00:00,  1.44s/it]


cannot convert float NaN to integer
saved to /mnt/data/bobae/ai_text_detection/dataset/ai/gptzero_ai.json...
